In [30]:
import sys
sys.path.append("../")

from mxnet import  nd, gluon, autograd
from d2l import linear_regression as lr
from mxnet.gluon import nn
from mxnet import init
from mxnet.gluon import loss as gloss


In [31]:
'''
    1. Generating Data Sets
'''

true_w  = nd.array([2, -3.4])
true_b =  4.2

features, labels = lr.synthetic_data(true_w, true_b, 1000)


In [32]:
'''
    2. Reading Data
'''
def load_array(data_arrays, batch_size, is_train = True):
    dataset =  gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle = is_train)

batch_size = 10
data_iter =  load_array((features, labels), batch_size)

for X, y in data_iter:
    print(X, y)
    break


[[-0.676097    0.9117826 ]
 [-0.62421465 -0.8773768 ]
 [-0.30135608 -0.0841791 ]
 [-1.8832594   1.5562055 ]
 [-0.04205222 -2.3422642 ]
 [ 0.05762542 -0.10484769]
 [-0.2864682   0.59069526]
 [-0.30959353  1.3222921 ]
 [ 0.18612005  0.60621804]
 [-1.5594455  -0.70880044]]
<NDArray 10x2 @cpu(0)> 
[-0.30947077  5.8963757   3.9195411  -4.5920196  12.195138    4.611757
  1.5967504  -0.90867716  2.7529073   3.5207562 ]
<NDArray 10 @cpu(0)>


In [33]:
'''
    3. Define the Model
'''
# We don't have to tell Gluon how may inputs go into this linear layer, Gluon will infer the number of inputs to each layer automatically
net = nn.Sequential()
# We only want to generate a single scalar
net.add(nn.Dense(1))

#Initialize Model Parameters
net.initialize(init.Normal(sigma= 0.01))
#Define the loss function
loss = gloss.L2Loss()

trainer = gluon.Trainer(net.collect_params(), "sgd", {"learning_rate" : 0.03})




In [34]:
'''
    4. Training
'''
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            # If we replace l = loss(output, y) with l = loss(output, y).mean(), we need to change trainer.step(batch_size) to trainer.step(1) accordingly. Why?
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l =  loss(net(features), labels)
    print("epoch %d loss: %f" % (epoch, l.mean().asnumpy()))
    
w = net[0].weight.data()
b = net[0].bias.data()

print("Error in estimating w", true_w.reshape(w.shape) - w)
print("Error in estimating b", true_b - b)
            

epoch 1 loss: 0.050262
epoch 2 loss: 0.005354
epoch 3 loss: 0.005234
Error in estimating w 
[[-0.00632286  0.00204611]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[-0.00538397]
<NDArray 1 @cpu(0)>
